## 0604 랜덤포레스트,트리들

In [1]:
!pip install xgboost

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support


In [2]:
!pip install --upgrade pip

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
Requirement already up-to-date: pip in /home/ec2-user/anaconda3/envs/pytorch_p27/lib/python2.7/site-packages (20.1.1)


In [3]:
# Data manipulation
import numpy as np
import pandas as pd

# Data pre-processing
from sklearn.preprocessing import StandardScaler as ss

# Dimensionality reduction
from sklearn.decomposition import PCA

#  Data splitting and model parameter search
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV


# Plotting
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import plot_importance


# 훈련 데이터와 테스트 데이터
아예 엑셀파일로 train, test, val 을 분류, x는 정규화 된 것을 가져왔음
비율--> 60:20:20

In [4]:
##data=pd.read_csv("./data/binary_winequalityN.csv")
X_train_std=pd.read_csv("./data/X_train_std.csv")
X_test_std=pd.read_csv("./data/X_test_std.csv")
X_val_std=pd.read_csv("./data/X_val_std.csv")
y_train=pd.read_csv("./data/y_train.csv")
y_test=pd.read_csv("./data/y_test.csv")
y_val=pd.read_csv("./data/y_val.csv")

print "X_train 크기: ", X_train_std.shape #(#(numbe) of data*75%, # of features)
print "y_train 크기: ", y_train.shape #x트레인 데이터에 대한 정답
print "X_val 크기: ", X_val_std.shape
print "y_val 크기: ", y_val.shape
print "X_test 크기: ", X_test_std.shape
print "y_test 크기: ", y_test.shape

X_train 크기:  (3896, 11)
y_train 크기:  (3896, 1)
X_val 크기:  (1299, 11)
y_val 크기:  (1299, 1)
X_test 크기:  (1299, 11)
y_test 크기:  (1299, 1)


---

#### 원본 

## 랜덤포레스트의 하이퍼파라미터
- n_estimators: 결정트리의 갯수를 지정, default 10
- min_samples_split: 노드를 분할하기 위한 최소한의 샘플 데이터, 과적합 제어시 사용, 작게 설정할 수록 과적합 가능성이 증가한다.
- min_samples_leaf: leaf노드가 되기 위한 최소한의 샘플데이터수, 과적합 제어용, 불균형한 데이터의 경우 특정 클래스의 데이터가 작을 수 있으므로 작게 설정해야한다.
- max_depth: 트리의 최대 깊이, 기본값 none(완벽하게 클래스 값이 결정될 때 까지 분할 or min_sample_split보다 작아질 때까지 분할), 깊이가 깊어지면 과적합될 수 있으므로 제어가 필요하다. 

출처 https://injo.tistory.com/30
참조 https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

### 랜덤포레스트 하이퍼파라미터 튜닝하기

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [39]:
params = { 'n_estimators' : [100],
           'max_depth' : [None, 6, 8, 10, 12],
           'min_samples_leaf' : [1, 2, 4, 8, 12, 18],
           'min_samples_split' : [2, 4, 8, 16, 20],
           'criterion': ['entropy','gini']
            }

# RandomForestClassifier 객체 생성 후 GridSearchCV 수행
rf_clf = RandomForestClassifier(random_state = 0, n_jobs = -1)
grid_cv = GridSearchCV(rf_clf, param_grid = params, cv = 3, n_jobs = -1)
grid_cv.fit(X_train_std, y_train)

print('The best hyperparameter: ', grid_cv.best_params_)
print('best prediction score: {:.6f}'.format(grid_cv.best_score_))

/home/ec2-user/anaconda3/envs/pytorch_p27/lib/python2.7/site-packages/sklearn/model_selection/_search.py:740: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


('The best hyperparameter: ', {'min_samples_split': 4, 'n_estimators': 100, 'criterion': 'entropy', 'max_depth': None, 'min_samples_leaf': 1})
best prediction score: 0.633727


In [65]:
random_forest_clf = RandomForestClassifier(n_estimators=100, min_samples_split=4,
                                           criterion='entropy',max_depth=None,
                                           min_samples_leaf=1)
random_forest_clf.fit(X_train_std,y_train)
pred=random_forest_clf.predict(X_val_std)
print('예측 정확도: {:.6f}'.format(accuracy_score(y_val,pred)))

/home/ec2-user/anaconda3/envs/pytorch_p27/lib/python2.7/site-packages/ipykernel/__main__.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


예측 정확도: 0.662818


### extra_trees 하이퍼파라미터 튜닝하기

In [41]:
from sklearn.ensemble import ExtraTreesClassifier

In [41]:

params = { 'n_estimators' : [100],
           'max_depth' : [None, 6, 8, 10, 12],
           'min_samples_leaf' : [1, 2, 4, 8, 12, 18],
           'min_samples_split' : [2, 4, 8, 16, 20],
           'criterion': ['entropy','gini']
            }

# ExtraTreesClassifier 객체 생성 후 GridSearchCV 수행
et_clf = ExtraTreesClassifier(random_state = 0, n_jobs = -1)
grid_cv = GridSearchCV(et_clf, param_grid = params, cv = 3, n_jobs = -1)
grid_cv.fit(X_train_std, y_train)

print('The best hyperparameter: ', grid_cv.best_params_)
print('best prediction score: {:.6f}'.format(grid_cv.best_score_))

/home/ec2-user/anaconda3/envs/pytorch_p27/lib/python2.7/site-packages/sklearn/model_selection/_search.py:740: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


('The best hyperparameter: ', {'min_samples_split': 4, 'n_estimators': 100, 'criterion': 'entropy', 'max_depth': None, 'min_samples_leaf': 1})
best prediction score: 0.642197


In [58]:
extra_trees_clf = ExtraTreesClassifier(n_estimators=100, min_samples_split=4,
                                           criterion='entropy',max_depth=None,
                                           min_samples_leaf=1)
extra_trees_clf.fit(X_train_std,y_train)
pred=extra_trees_clf.predict(X_val_std)
print('예측 정확도: {:.6f}'.format(accuracy_score(y_val,pred)))


/home/ec2-user/anaconda3/envs/pytorch_p27/lib/python2.7/site-packages/ipykernel/__main__.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


예측 정확도: 0.671286


In [69]:
extra_trees_clf = ExtraTreesClassifier(n_estimators=1000, min_samples_split=4,
                                           criterion='entropy',max_depth=None,
                                           min_samples_leaf=1)
extra_trees_clf.fit(X_train_std,y_train)
pred=extra_trees_clf.predict(X_val_std)
print('예측 정확도: {:.6f}'.format(accuracy_score(y_val,pred)))


/home/ec2-user/anaconda3/envs/pytorch_p27/lib/python2.7/site-packages/ipykernel/__main__.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


예측 정확도: 0.674365


In [70]:
pred=extra_trees_clf.predict(X_test_std)
print('예측 정확도: {:.6f}'.format(accuracy_score(y_test,pred)))

예측 정확도: 0.658199


### GradientBoostingClassifier

# 12시간 넘게 결과가 안나와서 종료함

In [7]:
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier

In [28]:
params = { 'n_estimators' : [10],
           'max_depth' : [None, 3, 6, 8, 16],
           'min_samples_leaf' : [1, 2, 4],
           'min_samples_split' : [2, 4],
           'criterion': ['friedman_mse','mse'], ##mae로하니까 결과가 오랫동안 안나와서 제외한
           'learning_rate': [0.2,0.1]
            }

# GradientBoostingClassifier 객체 생성 후 GridSearchCV 수행
gbc_clf = GradientBoostingClassifier(random_state = 0)
grid_cv = GridSearchCV(gbc_clf, param_grid = params, cv = 3)
grid_cv.fit(X_train_std, y_train)

print('The best hyperparameter: ', grid_cv.best_params_)
print('best prediction score: {:.6f}'.format(grid_cv.best_score_))

/home/ec2-user/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ec2-user/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ec2-user/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ec2-user/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s

/home/ec2-user/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ec2-user/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ec2-user/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ec2-user/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s

/home/ec2-user/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ec2-user/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ec2-user/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ec2-user/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s

/home/ec2-user/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ec2-user/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ec2-user/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ec2-user/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s

/home/ec2-user/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ec2-user/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ec2-user/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ec2-user/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s

/home/ec2-user/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ec2-user/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ec2-user/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ec2-user/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s

/home/ec2-user/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ec2-user/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ec2-user/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ec2-user/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s

/home/ec2-user/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ec2-user/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ec2-user/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ec2-user/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s

/home/ec2-user/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ec2-user/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ec2-user/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ec2-user/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s

KeyboardInterrupt: 

In [12]:
gradient_boost_clf = GradientBoostingClassifier(n_estimators=10, min_samples_split=4,
                                           max_depth=None,criterion='mse',
                                           min_samples_leaf=1,learning_rate=0.01)
gradient_boost_clf.fit(X_train_std,y_train)
pred=gradient_boost_clf.predict(X_val_std)
print('예측 정확도: {:.6f}'.format(accuracy_score(y_val,pred)))

/home/ec2-user/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


예측 정확도: 0.578137


In [13]:
gradient_boost_clf = GradientBoostingClassifier(n_estimators=10, min_samples_split=4,
                                           max_depth=None,criterion='mse',
                                           min_samples_leaf=1,learning_rate=0.1)
gradient_boost_clf.fit(X_train_std,y_train)
pred=gradient_boost_clf.predict(X_val_std)
print('예측 정확도: {:.6f}'.format(accuracy_score(y_val,pred)))

/home/ec2-user/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


예측 정확도: 0.581986


In [18]:
gradient_boost_clf = GradientBoostingClassifier(n_estimators=10, min_samples_split=4,
                                           max_depth=None,criterion='mse',
                                           min_samples_leaf=1,learning_rate=0.2)
gradient_boost_clf.fit(X_train_std,y_train)
pred=gradient_boost_clf.predict(X_val_std)
print('예측 정확도: {:.6f}'.format(accuracy_score(y_val,pred)))

/home/ec2-user/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


예측 정확도: 0.597383


In [35]:
gradient_boost_clf = GradientBoostingClassifier(n_estimators=10, min_samples_split=4,
                                           max_depth=None,criterion='friedman_mse',
                                           min_samples_leaf=1,learning_rate=0.2)
gradient_boost_clf.fit(X_train_std,y_train)
pred=gradient_boost_clf.predict(X_val_std)
print('예측 정확도: {:.6f}'.format(accuracy_score(y_val,pred)))

/home/ec2-user/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


예측 정확도: 0.603541


In [36]:
gradient_boost_clf = GradientBoostingClassifier(n_estimators=1000, min_samples_split=4,
                                           max_depth=None,criterion='friedman_mse',
                                           min_samples_leaf=1,learning_rate=0.2)
gradient_boost_clf.fit(X_train_std,y_train)
pred=gradient_boost_clf.predict(X_val_std)
print('예측 정확도: {:.6f}'.format(accuracy_score(y_val,pred)))

/home/ec2-user/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


예측 정확도: 0.614319


In [37]:
gradient_boost_clf = GradientBoostingClassifier(n_estimators=1000, min_samples_split=2,
                                           max_depth=None,criterion='friedman_mse',
                                           min_samples_leaf=1,learning_rate=0.2)
gradient_boost_clf.fit(X_train_std,y_train)
pred=gradient_boost_clf.predict(X_val_std)
print('예측 정확도: {:.6f}'.format(accuracy_score(y_val,pred)))

/home/ec2-user/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


예측 정확도: 0.597383


In [38]:
gradient_boost_clf = GradientBoostingClassifier(n_estimators=1000, min_samples_split=2,
                                           max_depth=None,criterion='friedman_mse',
                                           min_samples_leaf=1,learning_rate=0.1)
gradient_boost_clf.fit(X_train_std,y_train)
pred=gradient_boost_clf.predict(X_val_std)
print('예측 정확도: {:.6f}'.format(accuracy_score(y_val,pred)))

/home/ec2-user/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


예측 정확도: 0.590454


In [39]:
gradient_boost_clf = GradientBoostingClassifier(n_estimators=1000, min_samples_split=2,
                                           max_depth=None,criterion='friedman_mse',
                                           min_samples_leaf=1,learning_rate=0.01)
gradient_boost_clf.fit(X_train_std,y_train)
pred=gradient_boost_clf.predict(X_val_std)
print('예측 정확도: {:.6f}'.format(accuracy_score(y_val,pred)))

/home/ec2-user/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


예측 정확도: 0.576597


In [30]:
gradient_boost_clf = GradientBoostingClassifier(n_estimators=1000, min_samples_split=4,
                                           max_depth=3,criterion='friedman_mse',
                                           min_samples_leaf=1,learning_rate=0.2)
gradient_boost_clf.fit(X_train_std,y_train)
pred=gradient_boost_clf.predict(X_val_std)
print('예측 정확도: {:.6f}'.format(accuracy_score(y_val,pred)))

/home/ec2-user/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


예측 정확도: 0.589684


In [31]:
gradient_boost_clf = GradientBoostingClassifier(n_estimators=1000, min_samples_split=4,
                                           max_depth=3,criterion='friedman_mse',
                                           min_samples_leaf=1,learning_rate=0.1)
gradient_boost_clf.fit(X_train_std,y_train)
pred=gradient_boost_clf.predict(X_val_std)
print('예측 정확도: {:.6f}'.format(accuracy_score(y_val,pred)))

/home/ec2-user/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


예측 정확도: 0.583526


In [32]:
gradient_boost_clf = GradientBoostingClassifier(n_estimators=1000, min_samples_split=2,
                                           max_depth=3,criterion='friedman_mse',
                                           min_samples_leaf=1,learning_rate=0.1)
gradient_boost_clf.fit(X_train_std,y_train)
pred=gradient_boost_clf.predict(X_val_std)
print('예측 정확도: {:.6f}'.format(accuracy_score(y_val,pred)))

/home/ec2-user/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


예측 정확도: 0.580446


In [33]:
gradient_boost_clf = GradientBoostingClassifier(n_estimators=1000, min_samples_split=2,
                                           max_depth=3,criterion='friedman_mse',
                                           min_samples_leaf=1,learning_rate=0.2)
gradient_boost_clf.fit(X_train_std,y_train)
pred=gradient_boost_clf.predict(X_val_std)
print('예측 정확도: {:.6f}'.format(accuracy_score(y_val,pred)))

/home/ec2-user/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


예측 정확도: 0.588145


### HistGradientBoostingClassifier
히스토그램 기반

In [74]:
!pip install scikit-learn

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support


In [75]:
!python -m pip install scikit-learn --user --upgrade pip

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
     |████████████████████████████████| 5.5 MB 12.8 MB/s eta 0:00:01
Requirement already up-to-date: pip in /home/ec2-user/anaconda3/envs/pytorch_p27/lib/python2.7/site-packages (20.1.1)


In [77]:
%matplotlib inline

## 주피터에서 sklearn.experimental이 사용 안됨(sklearn issues통해 질문해놓은 상태) 

In [83]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

ImportError: No module named experimental

In [67]:
from sklearn.ensemble import HistGradientBoostingClassifier
params = { 'max_iter' : [100],
           'max_depth' : [None, 3, 6, 8],
           'min_samples_leaf' : [2, 4, 8,12,16,20,24],
           'loss': ['auto','binary_crossentropy','categorical_crossentropy'],
           'learning_rate': [0.2,0.1,0.05,0.02,0.01]
            }

# ExtraTreesClassifier 객체 생성 후 GridSearchCV 수행
hgb_clf = HistGradientBoostingClassifier(random_state = 0)
grid_cv = GridSearchCV(hgb_clf, param_grid = params, cv = 3)
grid_cv.fit(X_train_std, y_train)

print('The best hyperparameter: ', grid_cv.best_params_)
print('best prediction score: {:.6f}'.format(grid_cv.best_score_))

ImportError: cannot import name HistGradientBoostingClassifier